# Build a Smart Supply Chain (Warehouse Ops) Agent with Strands SDK, SAP BTP and Bedrock 

Welcome! In this tutorial, you'll learn how to build an intelligent warehouse operations agent that integrates with SAP's Warehouse and freight API to provide real-time inventory insights and decision support to deal with supply chain exceptions.

## Business Scenario: GlobalTech Manufacturing

**Company:** GlobalTech Manufacturing operates a high-tech distribution center (Warehouse 1750) handling critical electronic components.

**Products Examples (AI system can discover more automatically):**
- WM-AN01: Advanced Sensors
- WM-AN02: Control Units  
- WM-AN03: Power Modules
- WM-AN04: Communication Devices

**Challenge:** Warehouse managers need instant, intelligent responses to complex inventory questions like:
- "Do we have enough WM-AN02 units for the urgent ExampleCorp order?"
- "Which storage bins have the oldest inventory?"
- "Can we fulfill a 200-unit order across all product types?"
- "What's our current capacity utilization?"

**Solution:** An AI agent that connects to SAP APIs and provides human-like responses to complex supply chain queries.

---

## Prerequisites

1. SAP AI Core credentials in your `~/.aicore/config.json` file
2. Strands Agents SDK installed
3. SAP GenAI Hub SDK installed
4. Your actual SAP_S4HANA_PUBLIC_CLOUD_KEY

In [ ]:
# !pip install .

### Warehouse Ops Agent Strands System Design

In our design, we consume LLMs's through SAP's GenerativeAI Hub via SAP AI Core. This Warehouse Ops agent is built using a multi-agent architecture with the following key components:
1. **warehouse_agent** (Main Orchestrator)
Interprets user queries and orchestrates workflow by delegating API selection to selector_agent.
Executes OData queries via odata_caller and provides actionable insights with business context.

2. **selector_agent** (API Router Sub-Agent)
Analyzes queries and consults knowledgebase to identify the most appropriate SAP OData API.
Returns API recommendations with base URL and endpoint path for dynamic routing.
The `selector_agent` sub-agent is configured as a tool for the `warehouse_agent` following the Agent as a tool pattern.

3. **odata_caller** (Universal OData Tool)
Executes OData operations (GET, POST, PUT, DELETE) with automatic authentication.
Supports OData query parameters ($filter, $select, $orderby, $top) and structured error handling.

4. **knowledgebase** (OpenAPI Specification Repository)
Contains YAML/YML OpenAPI specs documenting available SAP OData APIs and endpoints.
Enables dynamic API discovery and easy extensibility without code changes.

#### Architecture Flow:
User query → **warehouse_agent** → **selector_agent** (consults **knowledgebase**) → **warehouse_agent** (uses **odata_caller**) → Business-friendly response

<div style="text-align:center">
    <img src="assets/riv25_arch.png" width="65%" />
</div>

## 1. Import Dependencies and Initialize Model

We'll use the Strands SDK with SAP GenAI Hub integration for our warehouse operations agent.

In [ ]:
from util.strands_bedrock_sap_genai_hub import SAPGenAIHubModel
from strands import Agent, tool
import os
from dotenv import load_dotenv
import getpass
from pathlib import Path
import yaml
from util.strands_tracer import *


# Load environment variables from .env file
load_dotenv()

# Prompt the user to securely input the API key if not already set in the environment
if not os.environ.get("SAP_S4HANA_PUBLIC_CLOUD_KEY"):
    os.environ["SAP_S4HANA_PUBLIC_CLOUD_KEY"] = getpass.getpass("SAP_S4HANA_PUBLIC_CLOUD_KEY:\n")


In [ ]:
from util.strands_bedrock_sap_genai_hub import SAPGenAIHubModel
from strands import Agent, tool

# Initialize the SAPGenAIHubModel with Claude Sonnet
# model = SAPGenAIHubModel(model_id="anthropic--claude-4.5-sonnet",
model = SAPGenAIHubModel(model_id="anthropic--claude-4-sonnet",
                                #  temperature = 0.3,
                                #  top_p = 1,
                                #  max_tokens = 25, 
                                #  stop_sequences = [ "blab" ],
                                )

# from strands.models import BedrockModel

# model = BedrockModel(
#     model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
#     region_name="us-east-1",
# )


## 2. Create Selector Sub-Agent
The Selector Sub-Agent acts as an intelligent API router that analyzes user queries and automatically determines which SAP OData API and endpoint should be used to fulfill the request. By loading and parsing OpenAPI specification files from the knowledgebase directory, it builds a comprehensive understanding of available APIs, their capabilities, endpoints, and base URLs. When given a user query, the selector agent evaluates the query against all available API specifications and provides a reasoned recommendation for which API and specific endpoint is most appropriate, including the correct sandbox base URL to use. This intelligent routing capability enables the main warehouse agent to dynamically work with multiple SAP APIs without hardcoded API selections, making the system more flexible and maintainable as new APIs are added to the knowledgebase.

In [ ]:

# Load YAML OpenAPI files from directory
def load_openapi_specs(path="./assets/knowledgebase"):
    specs = []
    for file in Path(path).glob("*.y*ml"):
        with open(file, "r") as f:
            try:
                data = yaml.safe_load(f)
                servers = data.get("servers", [])
                base_urls = []
                for s in servers:
                    url = s.get("url")
                    desc = s.get("description", "")
                    if url:
                        base_urls.append({"url": url, "description": desc})
                summary = {
                    "file": file.name,
                    "title": data.get("info", {}).get("title"),
                    "description": data.get("info", {}).get("description"),
                    "paths": list(data.get("paths", {}).keys()),
                    "base_urls": base_urls or [{"url": "/", "description": "Default"}]
                }
                specs.append(summary)
            except Exception as e:
                print(f"Error parsing {file}: {e}") 
    return specs

def specs_to_prompt_string(specs):
    prompt_parts = []
    for spec in specs:
        base_url_str = ", ".join(
            f"{url_info['url']} ({url_info['description']})" for url_info in spec['base_urls']
        )
        paths_str = ", ".join(spec['paths'])
        part = (
            f"API Spec: {spec['file']}\n"
            f"Title: {spec['title']}\n"
            f"Description: {spec['description']}\n"
            f"Base URLs: {base_url_str}\n"
            f"Endpoints: {paths_str}\n"
        )
        prompt_parts.append(part)
    return "\n---\n".join(prompt_parts)

# Usage example:
specs = load_openapi_specs()
specs_prompt_string = specs_to_prompt_string(specs)

SELECTOR_SYSTEM_PROMPT = f"""
You are an API selection subagent.
Given a user query and the following list of OpenAPI specs, each with a title, description, base URLs, and available endpoints:

{specs_prompt_string}

Your task is to identify which API and specific endpoint is most appropriate to fulfill the user query.
Provide clear reasoning for your choice, referencing the API spec details provided.
If no suitable API is found, explain why.

Make sure to reply with the sandbox BASE URL.

"""

# Strands subagent for selection
selector_agent = Agent(
    system_prompt=SELECTOR_SYSTEM_PROMPT,
    model=model
)

# query = "Get all inventory stock for item WM-AN02 "
query = "Get freight information for item WM-AN02 "

response = selector_agent(query)

print(response)


Let's now configure this sub-agent as a tool as we will be using the Agent as a tool pattern.

In [ ]:
@tool
def SelectorAPIAgentAsATool(query: str) -> str:
    """
    This analyzes available OpenAPI specs and selects the most appropriate
    API and endpoint based on user queries.

    Args:
        query: User query describing what they want to accomplish

    Returns:
        Agent response with API selection recommendation if successful, 
        error message if initialization fails
    """
    return selector_agent(query).message

## 3. Import the odata_tool from the util folder.
**odata_caller** (Universal OData Tool)
Executes OData operations (GET, POST, PUT, DELETE) with automatic authentication.
Supports OData query parameters ($filter, $select, $orderby, $top) and structured error handling.



In [ ]:
from util.odata_tool import odata_caller

## 4. Create the Warehouse Agent

Now let's create our intelligent warehouse operations agent with dynamic OData exploration capabilities.

In [ ]:
# Define the enhanced system prompt for our warehouse agent
warehouse_agent_prompt = """
You are an expert Warehouse Operations Manager for GlobalTech Manufacturing's Distribution Center (Warehouse 1750). 
You have access to real-time SAP warehouse data through dynamic OData API exploration capabilities.

CORE CAPABILITIES:
1. API Structure Exploration: Dynamically discover available data fields and entities
2. Product Discovery: Find available products without hardcoded assumptions
3. Dynamic Querying: Construct intelligent OData queries based on user needs

APPROACH TO PROBLEM SOLVING:
- You must start by using the SelectorAPIAgentAsATool to help you determine which OData API to call
- Next use the $metadata endpoint that to understand the API that SelectorAPIAgentAsATool provides  
- Use dynamic queries to discover information rather than making assumptions
- Leverage OData filtering, sorting, and selection to get precise answers
- You may need to do this multiple times. Always write what URL have constructed so user is informed.

PRODUCT KNOWLEDGE (can be expanded through discovery):
- WM-AN01: Advanced Sensors (high-precision electronic components)
- WM-AN02: Control Units (critical automation hardware)
- WM-AN03: Power Modules (electrical power management systems)
- WM-AN04: Communication Devices (networking and connectivity hardware)

COMMUNICATION STYLE:
- Be professional but conversational and succinct
- Explain your discovery process when exploring new data
- Provide specific, actionable insights with quantitative data
- Do not use emojis

When users ask questions:
1. First determine what data you need to answer the question
3. Feel free to use the odata_caller tool as many times as needed to get the right information.
3. Construct appropriate OData queries to get the specific information needed
4. Analyze the results and provide comprehensive, intelligent responses

Example for using the odata_caller tool:
```python
        odata_caller(
            base_url="",
            endpoint="WarehouseStockProducts",
            operation="get",
            odata_params={"$filter": "Product eq 'WM-AN02'"},
            auth_type="api_key",
            auth_env_var="SAP_S4HANA_PUBLIC_CLOUD_KEY"
        )
        ```

Focus on SAP S/4 HANA OData endpoints, warehouse management APIs, and supply chain operations.

Available tools:
- odata_caller: Universal OData tool for SAP API interactions with built-in authentication and query parameter support
- auth_token: Use os.getenv("SAP_S4HANA_PUBLIC_CLOUD_KEY") to get the API key
Example below:

# For SAP OData calls, use consistent headers
headers = {
    "APIKey": os.getenv("SAP_S4HANA_PUBLIC_CLOUD_KEY"),
    "Accept": "application/json", OR "application/xml" choose as needed 
    "DataServiceVersion": "2.0"
}

The odata_caller tool handles SAP-specific authentication automatically and provides comprehensive error handling and response formatting.

"""
        


# Create the enhanced warehouse operations agent
warehouse_agent = Agent(
    model=model,
    tools=[
        # ODataExecutorAgentAsATool,
        SelectorAPIAgentAsATool,
        odata_caller, # For OData REST API calls
        # explore_api_structure,
        # discover_available_products,
        # query_inventory_data,
        # analyze_inventory_patterns
    ],
    system_prompt=warehouse_agent_prompt
)

# # Quick test to verify the agent is working
# try:
#     test_response = warehouse_agent("Hello, can you introduce yourself and your capabilities?")
#     response_text = str(test_response)
    
#     print(f"\n Agent Test: {response_text[:150]}...")
# except Exception as e:
#     print(f"\n️ Agent test failed: {e}")
#     print("Agent created successfully, but test response had an issue.")

## 5. Demo Scenarios

Let's test our enhanced agent with realistic warehouse management scenarios. The agent will now dynamically explore the API to answer questions.

### Scenario 1: Complete Inventory Overview

In [ ]:
# Test inventory overview with dynamic discovery
response = warehouse_agent(
    "I need a complete overview of our current warehouse inventory. "
    "What products do we have, how much of each, and what's our overall stock situation?"
)
# print(response.message)

In [ ]:
# Get detailed trace with full message content
detailed_trace = trace_agent_execution_path(warehouse_agent, detailed=True, show_tool_results=True)

### Scenario 2: Urgent Order Fulfillment Check

In [ ]:
# Test urgent order scenario with dynamic discovery
warehouse_agent.messages = []
response = warehouse_agent(
    "We just received an urgent order from ExampleCorp for 200 units of WM-AN02 Control Units. "
    "Can we fulfill this order immediately? If yes, which storage bins should our picking team target first?"
)
# print(response.message)

In [ ]:
# Get detailed trace with full message content
detailed_trace = trace_agent_execution_path(warehouse_agent, detailed=True, show_tool_results=True)

### Scenario 3: Multi-Product Order Planning

In [ ]:
# Test complex multi-product scenario with dynamic queries
warehouse_agent.messages = []
response = warehouse_agent(
    "We're planning a large shipment for a new customer that needs: "
    "100 units of WM-AN01 sensors, 75 units of WM-AN03 power modules, and 50 units of WM-AN04 communication devices. "
    "Can we fulfill this entire order? What's our capacity after this shipment?"
)
# print(response.message)
# display_agent_metrics(response)

In [ ]:
# Get detailed trace with full message content
detailed_trace = trace_agent_execution_path(warehouse_agent, detailed=True, show_tool_results=True)

### Scenario 4: Warehouse Optimization Analysis

In [ ]:
# Test warehouse optimization with pattern analysis
warehouse_agent.messages = []
response = warehouse_agent(
    "Our operations team wants to optimize warehouse efficiency. "
    "Can you analyze our current storage utilization and recommend improvements? "
    "Also, which inventory should we prioritize for rotation based on age?"
)

## 6. Interactive Chat Mode

Try asking your own questions to the enhanced warehouse agent!

In [ ]:
# Interactive mode 
while True:
    user_question = input("\n Ask the Enhanced Warehouse Agent (or 'quit' to exit): ")
    if user_question.lower() in ['quit', 'exit', 'q', 'cancel']:
        break
    
    warehouse_agent.messages = []
    response = warehouse_agent(user_question)
    print(f"\n🤖 Agent Response:\n{response.message}")

## 7. Agent Execution Path Tracer

This helper function provides detailed insights into the agent's decision-making process, showing:
- The complete conversation flow
- Sub-agents invoked and their responses
- Tools used at each step
- Messages exchanged between agents
- Performance metrics for each interaction

### Trace the Warehouse Agent Execution

In [ ]:
# Trace the warehouse agent's execution path
trace_data = trace_agent_execution_path(warehouse_agent, detailed=False, show_tool_results=True)

In [ ]:
# Get tool usage statistics
tool_stats = get_tool_usage_stats(trace_data)

In [ ]:
# Export trace data for further analysis
# export_trace_to_json(trace_data, "warehouse_agent_trace.json")

### Detailed Trace with Full Content

For debugging or detailed analysis, you can get the full trace with complete message content:

In [ ]:
# Get detailed trace with full message content
detailed_trace = trace_agent_execution_path(warehouse_agent, detailed=True, show_tool_results=True)

In [ ]:
warehouse_agent.messages